In [ ]:
import pandas as pd
import os
import numpy as np
import re

In [ ]:
vote_files = os.listdir('../repro-votes-csvs/')
df = pd.DataFrame()
for vote_file in vote_files:
    
    print(vote_file)
    vote_df = pd.read_csv(f'../repro-votes-csvs/{vote_file}')
    options = np.array(vote_df.columns[1:].to_list())
    
    file = vote_file[:-4]
    year = '20' + vote_file[0:2]
    session = vote_file[vote_file.find('-') + 1:vote_file.find('_')]
    vote_id = vote_file[vote_file.find('_') + 1:vote_file.find('.')]

    number = re.search('[0-9]+', vote_id)[0]
    vote_type = vote_id[0:vote_id.find(number)]
    vote_chamber = vote_id[(vote_id.find(number) + len(number)):(vote_id.find(number) + len(number) + 1)]
    if vote_chamber == 'H':
        vote_chamber = 'House'
    else: 
        vote_chamber = 'Senate'
    names = []
    votes = []    
    
    for row in range(0, vote_df.shape[0]):
        
        row_list = vote_df.iloc[row, :].to_list()
        print(row_list)
        name = row_list.pop(0)
        if name[0:5] == 'TOTAL':
            continue
        ind = np.array([x in ['X', 'X*'] for x in row_list])
        vote = options[ind][0]
        votes.append(vote)
        names.append(name)

    vote_dict = {'file': file, 'year': year, 'session': session, 'legislation_type': vote_type, 
                 'legislation_number': number, 'voting_chamber': vote_chamber,
                 'legislator': names, 'vote': votes}
    df = pd.concat([df, pd.DataFrame(vote_dict)], axis = 0)

#df.to_csv('../repro-votes-reshaped.csv', index = False)
    

In [228]:
old_votes = pd.read_excel('../repro-votes.xlsx').dropna(axis = 0, how = 'all')
old_votes['Year'] = old_votes['Year'].astype('int')
old_votes['Legislation ID'] = old_votes['Legislation ID'].astype('int')
old_votes.drop_duplicates(subset = ['Year', 'Session', 'Legislation Type ', 'Legislation ID', 'Legislation Title'], inplace = True)
old_votes = old_votes.iloc[:, 0:5]
old_votes['Session'] = old_votes['Session'].str[5:]
old_votes.columns = ['year', 'session', 'legislation_type', 'legislation_number',
       'legislation_title']
#old_votes

In [248]:
df = pd.read_csv('../repro-votes-reshaped.csv')
df['year'] = df['year'].astype('int')
df['legislation_number'] = df['legislation_number'].astype('int')
df_joined = pd.merge(left = df, right = old_votes, how = 'left', 
                     on = ['year', 'session', 'legislation_type', 'legislation_number'])
df_joined['legislator'] = df_joined['legislator'].str.title()
df_joined['legislation_title'] = df_joined['legislation_title'].str.title()
df_joined.sort_values(['year', 'session', 'legislation_type', 'legislation_number', 'voting_chamber', 'legislator'],
                      ascending = [False, True, True, True, True, True], inplace = True)
df_joined = df_joined[['year', 'session', 'legislation_type', 'legislation_number', 'legislation_title', 'voting_chamber', 'legislator', 'vote']]
df_joined.columns = ['Year', 'Session', 'Legislation Type', 'Legislation Number', 'Legislation Title', 'Voting Chamber', 'Legislator', 'Vote']

df_joined.to_excel('../repro-votes-rehaped-titled.xlsx', index = False)